# 🌟 Entrenador Simple de Lora XL

❗ **Colab Premium no es necesario** pero es recommendado para datasets de tamaño grande. Idealmente para proyectos grandes se usaria una A100 y el `train_batch_size` al máximo.  



Basado en el trabajo de [Kohya-ss](https://github.com/kohya-ss/sd-scripts) y [Hollowstrawberry](https://github.com/hollowstrawberry/kohya-colab). ¡Gracias!

### ⭕ Renuncio de responsabilidad
El propósito de este documento es el estudio de la tecnología de inferencia con la IA.
Por favor lee y sigue las [Guías de Google Colab](https://research.google.com/colaboratory/faq.html) y sus [Términos de Servicio](https://research.google.com/colaboratory/tos_v3.html).

| |GitHub|🇬🇧 English|🇪🇸 Spanish|
|:--|:-:|:-:|:-:|
| 🏠 **Homepage** | [![GitHub](https://raw.githubusercontent.com/uYouUs/kohya-colab/main/assets/github.svg)](https://github.com/uYouUs/kohya-colab) | | |
| ⭐ **Lora Trainer** | [![GitHub](https://raw.githubusercontent.com/uYouUs/kohya-colab/main/assets/github.svg)](https://github.com/uYouUs/kohya-colab/blob/main/Lora_Trainer.ipynb) | [![Open in Colab](https://raw.githubusercontent.com/uYouUs/kohya-colab/main/assets/colab-badge.svg)](https://colab.research.google.com/github/uYouUs/kohya-colab/blob/main/Lora_Trainer.ipynb) | [![Abrir en Colab](https://raw.githubusercontent.com/uYouUs/kohya-colab/main/assets/colab-badge-spanish.svg)](https://colab.research.google.com/github/uYouUs/kohya-colab/blob/main/Spanish_Lora_Trainer.ipynb) |
| 🌟 **Simple XL Trainer** | [![GitHub](https://raw.githubusercontent.com/uYouUs/kohya-colab/main/assets/github.svg)](https://github.com/uYouUs/kohya-colab/blob/main/Simple_XL_Trainer.ipynb) | [![Open in Colab](https://raw.githubusercontent.com/uYouUs/kohya-colab/main/assets/colab-badge.svg)](https://colab.research.google.com/github/uYouUs/kohya-colab/blob/main/Simple_XL_Trainer.ipynb) |  |
| 🌟 **XL Lora Trainer** | [![GitHub](https://raw.githubusercontent.com/uYouUs/kohya-colab/main/assets/github.svg)](https://github.com/uYouUs/kohya-colab/blob/main/Lora_Trainer_XL.ipynb) | [![Open in Colab](https://raw.githubusercontent.com/uYouUs/kohya-colab/main/assets/colab-badge.svg)](https://colab.research.google.com/github/uYouUs/kohya-colab/blob/main/Lora_Trainer_XL.ipynb) |  |

In [ ]:
import os
import re
import toml
import pathlib
import threading
import zipfile
from time import time
from IPython.display import Markdown, display

# These carry information from past executions
if "dependencies_installed" not in globals():
  dependencies_installed = False
if "diffusers_model" not in globals():
  diffusers_model = None

# These may be set by other cells, some are legacy
if "custom_dataset" not in globals():
  custom_dataset = None
if "override_dataset_config_file" not in globals():
  override_dataset_config_file = None
if "continue_from_lora" not in globals():
  continue_from_lora = ""
if "override_config_file" not in globals():
  override_config_file = None

COLAB = True
SOURCE = "https://github.com/uYouUs/sd-scripts"
BRANCH = "simple"
COMMIT = None
try:
  LOWRAM = int(next(line.split()[1] for line in open('/proc/meminfo') if "MemTotal" in line)) / (1024**2) < 15
except:
  LOWRAM = False

#@title ## 🚩 Empieza Aquí

#@markdown ### ▶️ Configuración

#@markdown El nombre de su proyecto se utilizará para encontrar el archivo zip en su Google Drive y para los nombres de Lora. No se permiten espacios.<p>
#@markdown Asegúrese de que el archivo .zip de su dataset esté en su Google Drive fuera de cualquier carpeta y tenga un nombre como en el siguiente campo.
project_name = "" #@param {type:"string"}
project_name = project_name.strip()

zip = "/content/drive/MyDrive/" + project_name + ".zip"

output_location = "/content/drive/MyDrive/Loras/"
#@markdown Decide el modelo base que se descargará y utilizará para la entrenar.
training_model = "Illustrious" #@param ["Illustrious", "Pony", "AnimagineXL3.0", "NoobAI Eps", "NoobAI V-Pred", "SDXL 1.0"]
vae_file= "stabilityai/sdxl-vae"
vpred = False

if "Pony" in training_model:
  diffusers_model = "uYouUs/PonyV6"
elif "Animagine" in training_model:
  diffusers_model ="cagliostrolab/animagine-xl-3.0"
elif "Illustrious" in training_model:
  diffusers_model = "uYouUs/IllustriousV01"
elif "NoobAI Eps" in training_model:
  diffusers_model = "Laxhar/noobai-XL-1.1"
elif "NoobAI V-Pred" in training_model:
  diffusers_model = "Laxhar/noobai-XL-Vpred-1.0"
  vpred = True
else:
  diffusers_model = "stabilityai/stable-diffusion-xl-base-1.0"


#@markdown ### ▶️ Procesamiento

caption_extension = ".txt"
#@markdown Mezclar las etiquetas ayuda al aprendizaje. Las etiquetas de activación van al inicio de cada archivo de texto y no se mezclarán.<p>
activation_tags = "1" #@param [0,1,2,3]
keep_tokens = int(activation_tags)

#@markdown ### ▶️ Épocas <p>

#@markdown Elige cuántas épocas guardar. Por defecto, se entrenarán 15. Puedes conservar todas o solo las últimas.<p>
max_train_epochs = 10
#@markdown Guardar más épocas te permitirá comparar mejor el progreso de tu Lora a cambio de necesitar más espacio de almacenamiento.
save_every_n_epochs = 1
keep_only_last_n_epochs = 15 #@param {type:"number"}
if not save_every_n_epochs:
  save_every_n_epochs = max_train_epochs
if not keep_only_last_n_epochs:
  keep_only_last_n_epochs = max_train_epochs

#@markdown ### ▶️ Estructura
lora_type = "LoRA"

#@markdown Abajo estan los valores recomendados para las siguientes configuraciones:

#@markdown | type | network_dim |
#@markdown | :---: | :---: |
#@markdown | Regular LoRA | 8 |


#@markdown Un dim mayor crea a una Lora más grande, pero no siempre es mejor. Aumenta solo si usas un dataset grande.
network_dim = 8 #@param {type:"slider", min:1, max:32, step:1}

#@markdown ### ▶️ Avanzado (Opcional)

#@markdown Déjalo como está a menos que:
#@markdown 1. Si tiene errores de memoria con datasets grandes, reduzca este valor.
#@markdown 2. Estás usando una GPU pagada, con más memoria puedes aumentar esto.
train_batch_size = 5 #@param {type:"slider", min:1, max:16, step:1}
#@markdown Si estás usando una A100, deberias usar bf16.
mixed_precision = "fp16" #@param ["bf16", "fp16"]

#@markdown Este valor cambia el tiempo de entrenamiento. Si sientes que las Loras no están completamente entrenadas, puedes aumentar este tiempo.<p>
#@markdown Este valor multiplicado por 100 equivale a una estimación de los pasos de entrenamiento que se realizarán. Se recomienda un valor de 14.
train_time = 14 #@param {type:"slider", min:1, max:40, step:1}
real_step_estimate = train_time * 100

#@markdown ### ▶️ Generar imagen de muestra (Opcional)

#@markdown Genera una imagen de muestra con la Lora después de cada época guardada. Esto permite supervisar el entrenamiento.<p>
#@markdown Las imágenes se guardarán en `GDrive/Loras/project_name/output/samples`<p>
#@markdown Agrega opciones de generación opcionales al final usando `--w {width} --h {height} --d {seed} --s {steps} --l {cfg}`<p>
#@markdown Prompt de ejemplo: `1girl, blonde hair, smiling --w 1024 --h 1024 --d 173371316 --s 20 --l 5`<p>
sample_prompt = "" #@param {type:"string"}

#@markdown ### ▶️ Listo
#@markdown Ahora puedes correr esta celda apretando el botón circular a la izquierda. ¡Buena suerte! <p>

#Settings
optimizer_args = "weight_decay=0.1 betas=[0.9,0.99]"
optimizer_args = [a.strip() for a in optimizer_args.split(' ') if a]
lr_warmup_ratio = 0.05
lr_warmup_steps = 0
num_repeats = 1

# 👩‍💻 Cool code goes here

root_dir = "/content" if COLAB else pathlib.Path.home() / "Loras"
deps_dir = os.path.join(root_dir, "deps")
repo_dir = os.path.join(root_dir, "kohya-trainer")

main_dir      = os.path.join(root_dir, "Loras") if COLAB else root_dir
log_folder    = os.path.join(main_dir, "_logs")
config_folder = os.path.join(main_dir, project_name)
images_folder = os.path.join(main_dir, project_name, "dataset")
output_folder = os.path.join(output_location, project_name, "output")

config_file = os.path.join(output_location, project_name, "training_config.toml")
dataset_config_file = os.path.join(output_location, project_name, "dataset_config.toml")
accelerate_config_file = os.path.join(repo_dir, "accelerate_config/config.yaml")

def install_dependencies():
  print("Cloning Kohya")
  os.chdir(root_dir)
  !git clone {SOURCE} {repo_dir}
  os.chdir(repo_dir)
  if BRANCH:
    !git checkout {BRANCH}
  if COMMIT:
    !git reset --hard {COMMIT}
  !wget https://raw.githubusercontent.com/uYouUs/kohya-colab/main/train_network_xl_wrapper.py -q -O train_network_xl_wrapper.py
  !wget https://raw.githubusercontent.com//uYouUs/kohya-colab/main/dracula.py -q -O dracula.py

  !apt -y update -qq
  !apt -y install aria2 -qq

  #!pip install accelerate==1.2.1 opencv-python==4.10.0.84 einops==0.8.0 \ #debug, slows down startup if active
  !pip install bitsandbytes==0.45.1 pytorch-lightning==1.9.0 voluptuous==0.13.1 \
    invisible-watermark==0.2.0 prodigyopt==1.0.0 \
    dadaptation==3.1 lion-pytorch==0.1.2 ftfy==6.1.1
    # toml==0.10.2 safetensors pygments wandb imagesize==1.4.1 #debug
  !pip install -e .

  # patch kohya for minor stuff
  if LOWRAM:
    !sed -i "s@cpu@cuda@" library/model_util.py
  !sed -i 's/from PIL import Image/from PIL import Image, ImageFile\nImageFile.LOAD_TRUNCATED_IMAGES=True/g' library/train_util.py # fix truncated jpegs error
  !sed -i 's/{:06d}/{:02d}/g' library/train_util.py # make epoch names shorter
  !sed -i 's/"." + args.save_model_as)/"-{:02d}.".format(saved_count+1) + args.save_model_as)/g' train_network.py # name of the last epoch will match the rest

  from accelerate.utils import write_basic_config
  if not os.path.exists(accelerate_config_file):
    write_basic_config(save_location=accelerate_config_file)

  os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
  os.environ["BITSANDBYTES_NOWELCOME"] = "1"
  os.environ["SAFETENSORS_FAST_GPU"] = "1"

def validate_dataset():
  global lr_warmup_steps, lr_warmup_ratio, caption_extension, keep_tokens
  supported_types = (".png", ".jpg", ".jpeg", ".webp", ".bmp")

  print("\n💿 Checking dataset...")
  if not project_name.strip() or any(c in project_name for c in " .()\"'\\/"):
    print("💥 Error: Please choose a valid project name.")
    return

  # Find the folders and files
  if custom_dataset:
    try:
      datconf = toml.loads(custom_dataset)
      datasets = [d for d in datconf["datasets"][0]["subsets"]]
    except:
      print(f"💥 Error: Your custom dataset is invalid or contains an error! Please check the original template.")
      return
    reg = [d.get("image_dir") for d in datasets if d.get("is_reg", False)]
    datasets_dict = {d["image_dir"]: d["num_repeats"] for d in datasets}
    folders = datasets_dict.keys()
    files = [f for folder in folders for f in os.listdir(folder)]
    images_repeats = {folder: (len([f for f in os.listdir(folder) if f.lower().endswith(supported_types)]), datasets_dict[folder]) for folder in folders}
  else:
    reg = []
    folders = [images_folder]
    files = os.listdir(images_folder)
    images_repeats = {images_folder: (len([f for f in files if f.lower().endswith(supported_types)]), num_repeats)}

  # Validation
  for folder in folders:
    if not os.path.exists(folder):
      print(f"💥 Error: The folder {folder.replace('/content/drive/', '')} doesn't exist.")
      return
  for folder, (img, rep) in images_repeats.items():
    if not img:
      print(f"💥 Error: Your {folder.replace('/content/drive/', '')} folder is empty.")
      return
  test_files = []
  for f in files:
    if not f.lower().endswith((caption_extension, ".npz")) and not f.lower().endswith(supported_types):
      print(f"💥 Error: Invalid file in dataset: \"{f}\". Aborting.")
      return
    for ff in test_files:
      if f.endswith(supported_types) and ff.endswith(supported_types) \
          and os.path.splitext(f)[0] == os.path.splitext(ff)[0]:
        print(f"💥 Error: The files {f} and {ff} cannot have the same name. Aborting.")
        return
    test_files.append(f)

  if caption_extension and not [txt for txt in files if txt.lower().endswith(caption_extension)]:
    caption_extension = ""
  if continue_from_lora and not (continue_from_lora.endswith(".safetensors") and os.path.exists(continue_from_lora)):
    print(f"💥 Error: Invalid path to existing Lora. Example: /content/drive/MyDrive/Loras/example.safetensors")
    return

  # Pretty stuff

  pre_steps_per_epoch = sum(img*rep for (img, rep) in images_repeats.values())
  steps_per_epoch = pre_steps_per_epoch/train_batch_size
  total_steps = int(max_train_epochs*steps_per_epoch)
  lr_warmup_steps = int(total_steps*lr_warmup_ratio)

  for folder, (img, rep) in images_repeats.items():
    print("📁"+folder.replace("/content/drive/", "") + (" (Regularization)" if folder in reg else ""))
    print(f"📈 Found {img} images.")

  if total_steps > 10000:
    print("💥 Error: Your total steps are too high. You probably made a mistake or dataset is too big. Aborting...")
    return

  return True

def create_config():
  global dataset_config_file, config_file

  if override_config_file:
    config_file = override_config_file
    print(f"\n⭕ Using custom config file {config_file}")
  else:
    config_dict = {
      "network_arguments": {
        "unet_lr": 1,
        "text_encoder_lr": 1,
        "network_dim": network_dim,
        "network_alpha": network_dim,
        "network_module": "networks.lora",
        "network_args": None,
        "network_weights": continue_from_lora if continue_from_lora else None
      },
      "optimizer_arguments": {
        "learning_rate": 1,
        "lr_scheduler": "cosine",
        "lr_scheduler_num_cycles": None,
        "lr_scheduler_power": None,
        "lr_warmup_steps": lr_warmup_steps,
        "optimizer_type": "Prodigy",
        "optimizer_args": optimizer_args if optimizer_args else None,
      },
      "training_arguments": {
        "pretrained_model_name_or_path": diffusers_model,
        "vae": vae_file,
        "max_train_epochs": max_train_epochs,
        "train_batch_size": train_batch_size,
        "seed": 42,
        "max_token_length": 225,
        "sdpa": True,
        "min_snr_gamma": 8.0,
        "lowram": LOWRAM,
        "no_half_vae": True,
        "gradient_checkpointing": True,
        "gradient_accumulation_steps": 1,
        "max_data_loader_n_workers": 3,
        "persistent_data_loader_workers": True,
        "mixed_precision": mixed_precision,
        "full_bf16": mixed_precision == "bf16",
        "cache_latents": True,
        "cache_latents_to_disk": True,
        "cache_text_encoder_outputs": False,
        "min_timestep": 0,
        "max_timestep": 1000,
        "prior_loss_weight": 1.0,
        "multires_noise_iterations": 6,
        "multires_noise_discount": 0.3,
        "v_parameterization": vpred,
        "scale_v_pred_loss_like_noise_pred": vpred,
        "zero_terminal_snr": vpred,
        "real_step_estimate": real_step_estimate,
        "real_epoch": 15,
      },
      "saving_arguments": {
        "save_precision": "fp16",
        "save_model_as": "safetensors",
        "save_every_n_epochs": save_every_n_epochs,
        "save_last_n_epochs": keep_only_last_n_epochs,
        "output_name": project_name,
        "output_dir": output_folder,
        "log_prefix": project_name,
        "logging_dir": log_folder,
        "sample_prompts": prompt_file if sample_prompt else None,
        "sample_every_n_epochs": save_every_n_epochs if sample_prompt else None,
        "sample_sampler": "euler_a" if sample_prompt else None,
      }
    }

    for key in config_dict:
      if isinstance(config_dict[key], dict):
        config_dict[key] = {k: v for k, v in config_dict[key].items() if v is not None}

    with open(config_file, "w") as f:
      f.write(toml.dumps(config_dict))
    print(f"\n📄 Config saved to {config_file}")

  if override_dataset_config_file:
    dataset_config_file = override_dataset_config_file
    print(f"⭕ Using custom dataset config file {dataset_config_file}")
  else:
    dataset_config_dict = {
      "general": {
        "resolution": 1024,
        "shuffle_caption": True,
        "keep_tokens": keep_tokens,
        "flip_aug": False,
        "caption_extension": caption_extension,
        "enable_bucket": True,
        "bucket_no_upscale": False,
        "bucket_reso_steps": 64,
        "min_bucket_reso": 256,
        "max_bucket_reso": 4096,
      },
      "datasets": toml.loads(custom_dataset)["datasets"] if custom_dataset else [
        {
          "subsets": [
            {
              "num_repeats": num_repeats,
              "image_dir": images_folder,
              "class_tokens": None
            }
          ]
        }
      ]
    }

    for key in dataset_config_dict:
      if isinstance(dataset_config_dict[key], dict):
        dataset_config_dict[key] = {k: v for k, v in dataset_config_dict[key].items() if v is not None}

    with open(dataset_config_file, "w") as f:
      f.write(toml.dumps(dataset_config_dict))
    print(f"📄 Dataset config saved to {dataset_config_file}")

def getModel():
  global diffusers_model
  #!pip install transformers==4.47.1 diffusers==0.32.2 jax==0.4.33 jaxlib==0.4.33 huggingface_hub==0.27.1 flax==0.10.2
  from huggingface_hub import snapshot_download
  from huggingface_hub.utils import disable_progress_bars
  disable_progress_bars() # Disable model download progress bars to clean up output from threading mess
  snapshot_download(repo_id=vae_file, allow_patterns=["*model.safetensors", "*.json"]) # Download Vae
  try:
    print("\n🔄 Getting Diffuser model...")
    snapshot_download(repo_id=diffusers_model, allow_patterns=["*model.safetensors", "*.json", "*.txt"]) # Much faster but runs into chance of downloading unnecessary files, slowing it down in rare cases.
  except:
    raise ValueError(f"\nInvalid Diffuser Model {diffusers_model}")

def unzipset():
  if COLAB and not os.path.exists('/content/drive'):
    from google.colab import drive
    print("\n📂 Connecting to Google Drive...")
    drive.mount('/content/drive')

  print("\nUnzipping")
  os.makedirs(images_folder, exist_ok=True)
  with zipfile.ZipFile(zip, 'r') as f:
    f.extractall(images_folder)
  print("\n✅ Done unzipping")

def generateSample(): # Generate required files for sample generation
  global output_folder, sample_prompt, prompt_file
  if sample_prompt:
    prompt_file = os.path.abspath(os.path.join(output_folder, "sample"))
    if not os.path.exists(prompt_file): # Make sample directory so prompt file creation does not fail.
      os.mkdir(prompt_file)
    prompt_file = os.path.join(prompt_file, "prompt.txt")
    with open(prompt_file, "w") as f:
      f.write(sample_prompt)

def thInstallDep():
  global dependencies_installed
  print("\n🏭 Installing dependencies...\n")
  t0 = time()
  install_dependencies()
  t1 = time()
  dependencies_installed = True
  print(f"\n✅ Installation finished in {int(t1-t0)} seconds. \nWaiting on Model...\n")

def main():
  global dependencies_installed, sample_prompt
  threadList = []
  # Download model with a thread
  thr1 = threading.Thread(target = getModel)
  threadList.append(thr1)
  thr1.start()

  if not dependencies_installed: # Begin installing dependencies while user gives gdrive permissions
    thr2 = threading.Thread(target = thInstallDep)
    threadList.append(thr2)
    thr2.start()
  else:
    print("\n✅ Dependencies already installed.")

  if custom_dataset is None:  # Unzip dataset with a thread
    thr3 = threading.Thread(target = unzipset)
    threadList.append(thr3)
    thr3.start()

  elif COLAB and not os.path.exists('/content/drive'): # Request drive access for dataset validatation if we did not unzip it
    from google.colab import drive
    print("\n📂 Connecting to Google Drive...")
    drive.mount('/content/drive')

  for th in threadList: # Make sure model and or set are done before continuing
    th.join()


  for dir in (main_dir, deps_dir, repo_dir, log_folder, images_folder, output_folder, config_folder):
    os.makedirs(dir, exist_ok=True)

  if not validate_dataset():
    return

  generateSample() # Make sure sample file exists prior to creating config file
  create_config()

  print("\n⭐ Starting trainer...\n")
  os.chdir(repo_dir)

  !accelerate launch --quiet --config_file={accelerate_config_file} --num_cpu_threads_per_process=2 --num_processes=2 train_network_xl_wrapper.py --dataset_config={dataset_config_file} --config_file={config_file}

  if not get_ipython().__dict__['user_ns']['_exit_code']:
    display(Markdown("### ✅ Done! [Go download your Lora from Google Drive](https://drive.google.com/drive/my-drive)\n"
                     "### There will be several files, you should try the latest version (the file with the largest number next to it)"))

main()


## *️⃣ Extras

Puedes ejecutarlas antes de comenzar el entrenamiento.

In [ ]:
#@markdown ### Continuar

#@markdown Aquí puedes escribir el lugar en tu Google Drive para cargar una Lora existente para continuar con el entrenamiento.<p>
#@markdown **Advertencia:** No es lo mismo que una larga sesión de entrenamiento. Las épocas empiezan desde cero y puede tener resultados peores.
continue_from_lora = "" #@param {type:"string"}
if continue_from_lora and not continue_from_lora.startswith("/content/drive/MyDrive"):
  import os
  continue_from_lora = os.path.join("/content/drive/MyDrive", continue_from_lora)


### 📚 Múltiples carpetas
**Para usuarios avanzados:** Antes de iniciar el entrenamiento, puedes editar y correr la celda aquí abajo, la cual tiene un ejemplo para definir tus propias carpetas de imágenes con diferentes repeticiones. Para agregar más carpetas, simplemente copie y pegue las secciones que comienzan con `[[datasets.subsets]]`.

El nombre de la carpeta principal con el nombre del proyecto de la celda principal será ignorado.

Puedes hacer que una carpeta contenga imágenes de regularización con la frase `is_reg = true`
También puedes poner distintos `keep_tokens`, `flip_aug`, etc.

Al utilizar Google Drive para los datasets, el lugar predeterminado es:
`content/drive/MyDrive/Loras/example/dataset/normal_images`

Para descomprimir el .zip en la colab, la ubicación predeterminada es:
`content/Loras/example/dataset/normal_images`

<b>Advertencia:</b> Usar un número de repeticiones personalizado distinto de 1 probablemente causará problemas extraños con la automatización y puede reducir el rendimiento. Si es posible, manténgalo en num_repeats=1 y deja que lo determine automáticamente.

In [ ]:
custom_dataset = """
[[datasets]]

[[datasets.subsets]]
image_dir = "/content/drive/MyDrive/Loras/example/dataset/good_images"
num_repeats = 1

[[datasets.subsets]]
image_dir = "/content/drive/MyDrive/Loras/example/dataset/normal_images"
num_repeats = 1

"""

In [ ]:
custom_dataset = None